In [30]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import clear_output

import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)

from parameters import *
from autoencoder import *
from utils import *

In [31]:
current_ds = pd.read_csv(TRAIN_DIR['current'])
old_ds = pd.read_csv(TRAIN_DIR['old'])
hewlett = pd.read_csv(TRAIN_DIR['hewlett'])

label_X = current_ds['full_text'].to_list()
unlabled_X = old_ds['texts'].to_list()
unlabled_X.extend(hewlett['essay'].to_list())
y = current_ds.iloc[:, 2].values

In [32]:
class LSTMPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size, self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, all_hidden_states):
        ## forward
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out, _ = self.lstm(hidden_states, None)
        out = self.dropout(out[:, -1, :])
        return out

In [33]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [34]:
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

embeddings = []

for i in tqdm(range(0, len(label_X), 16)):
    tail = i + 16 if i+16 < len(label_X) else len(label_X)
    
    features = tokenizer.batch_encode_plus(
        label_X[i:tail],
        add_special_tokens=True,
        padding='max_length',
        max_length=max_seq_length,
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True
    )
    features = features.to(device)

    with torch.no_grad():
        outputs = model(features['input_ids'], features['attention_mask'])
    
    all_hidden_states = torch.stack(outputs[2])
    
    hiddendim_lstm = 256
    pooler = LSTMPooling(config.num_hidden_layers, config.hidden_size, hiddendim_lstm)
    pooler = pooler.to(device)
    embeddings.append(pooler(all_hidden_states).cpu().detach().numpy())


100%|██████████| 245/245 [00:22<00:00, 11.02it/s]


In [35]:
embeddings_np = np.concatenate(embeddings, axis=0)
embeddings_np.shape

(3911, 256)

In [36]:
np.save('labeled_data.npy', embeddings_np)

In [37]:
embeddings_2 = []

for i in tqdm(range(0, len(unlabled_X), 16)):
    tail = i + 16 if i+16 < len(unlabled_X) else len(unlabled_X)
    
    features = tokenizer.batch_encode_plus(
        unlabled_X[i:tail],
        add_special_tokens=True,
        padding='max_length',
        max_length=max_seq_length,
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True
    )
    features = features.to(device)

    with torch.no_grad():
        outputs = model(features['input_ids'], features['attention_mask'])
    
    all_hidden_states = torch.stack(outputs[2])
    
    hiddendim_lstm = 256
    pooler = LSTMPooling(config.num_hidden_layers, config.hidden_size, hiddendim_lstm)
    pooler = pooler.to(device)
    embeddings_2.append(pooler(all_hidden_states).cpu().detach().numpy())


100%|██████████| 1786/1786 [02:47<00:00, 10.68it/s]


In [38]:
embeddings_2_np = np.concatenate(embeddings_2, axis=0)
embeddings_2_np.shape

(28570, 256)

In [39]:
np.save('unlabeled_data.npy', embeddings_2_np)

In [40]:
np.save('labels.npy', y)